# LiH molecule

## Origin of this method: Low rank decomposition of the Coulomb operator
 "Low rank representations for quantum simulation of electronic structure"
Mario Motta, Erika Ye, Jarrod R. McClean, Zhendong Li, Austin J. Minnich, Ryan Babbush, Garnet Kin-Lic Chan
https://arxiv.org/abs/1808.02625

The code is adapted from OpenFermion-Cirq Tutorial III: Low rank, arbitrary basis molecular simulations https://github.com/quantumlib/OpenFermion-Cirq/blob/master/examples/tutorial_3_arbitrary_basis_trotter.ipynb

In Tutorial III both of those techniques are combined, along with some insights from electronic structure,
to simulate a Trotter step under the arbitrary basis two-body operator as
$$
\prod_{\ell=0}^{L-1} R_\ell \exp\left(-i\sum_{pq} f_{\ell p} f_{\ell q} a^\dagger_p a_p a^\dagger_q a_q\right) R_\ell^\dagger
$$
where we note that the operator in the exponential take the form of a diagonal Coulomb operator. Since we can implement the $R_\ell$ circuits in $O(N)$ depth (see Tutorial I) and we can implement Trotter steps under diagonal Coulomb operators in $O(N)$ layers of gates (see Tutorial II) we see that we can implement Trotter steps under arbitrary basis electronic structure Hamiltionians in $O(L N) = O(N^2)$ depth, and all on a linearly connected device.

## Example implementation: Trotter steps of LiH in molecular orbital basis

We will now use these techniques to implement Trotter steps for an actual molecule. We will focus on LiH at equilibrium geometry, since integrals for that system are provided with every OpenFermion installation. However, by installing [OpenFermion-PySCF](https://github.com/quantumlib/OpenFermion-PySCF) or [OpenFermion-Psi4](https://github.com/quantumlib/OpenFermion-Psi4) one can use these techniques for any molecule at any geometry. We will generate LiH in an active space consisting of 4 qubits. First, we obtain the Hamiltonian as an InteractionOperator.

In [1]:
import openfermion

# Set Hamiltonian parameters for LiH simulation in active space.
diatomic_bond_length = 1.45
geometry = [('Li', (0., 0., 0.)), ('H', (0., 0., diatomic_bond_length))]
basis = 'sto-3g'
multiplicity = 1
active_space_start = 1
active_space_stop = 3

# Generate and populate instance of MolecularData.
molecule = openfermion.MolecularData(geometry, basis, multiplicity, description="1.45")
molecule.load()

# Get the Hamiltonian in an active space.
molecular_hamiltonian = molecule.get_molecular_hamiltonian(
    occupied_indices=range(active_space_start),
    active_indices=range(active_space_start, active_space_stop))

print("Molecular Hamiltonian with 1 constant and {} 1-body and {} 2-body tensor terms"
        .format(molecular_hamiltonian.one_body_tensor.size, 
                molecular_hamiltonian.two_body_tensor.size))

# obtain the Hamiltonian as matrix
hamiltonian_sparse = openfermion.get_sparse_operator(molecular_hamiltonian)
LiH_matrix = hamiltonian_sparse.todense()
print("Hamiltonian matrix as {} from which {} are not null"
      .format( LiH_matrix.shape, hamiltonian_sparse.nnz))

# solve for eigenvalues by matrix algorithms
from scipy.linalg import eigh
eigenvalues , eigenvectors = eigh(LiH_matrix)
print("Eigenvalues  (Energies)\n", eigenvalues.round(6))
print("Eigenvectors (Orbitals)\n", eigenvectors.real.round(1))

Molecular Hamiltonian with 1 constant and 16 1-body and 256 2-body tensor terms
Hamiltonian matrix as (16, 16) from which 36 are not null
Eigenvalues  (Energies)
 [-7.862773 -7.783396 -7.783396 -7.714057 -7.714057 -7.714057 -7.700476
 -7.569985 -7.569985 -7.512    -7.512    -7.364817 -7.151525 -7.130407
 -7.130407 -6.769813]
Eigenvectors (Orbitals)
 [[ 0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   1. ]
 [ 0.   0.   0.   0.   0.   0.   0.   0.   0.1  0.   0.   0.   0.   1.
   0.1  0. ]
 [ 0.   0.   0.   0.   0.   0.   0.   0.1 -0.   0.   0.   0.   0.  -0.1
   1.   0. ]
 [ 0.   0.   0.   0.   0.   0.  -0.1  0.   0.   0.   0.   0.   1.   0.
   0.   0. ]
 [ 0.   0.   0.   0.   0.   0.   0.  -0.1 -1.   0.   0.   0.   0.   0.1
   0.   0. ]
 [ 0.   0.   0.   1.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0. ]
 [ 0.  -0.  -0.   0.   0.7  0.  -0.7  0.   0.  -0.  -0.   0.  -0.1  0.
   0.   0. ]
 [ 0.  -0.  -0.   0.   0.  -0.   0.   0.   0.  -0.  -1.   

We are not yet aiming for chemical accuracy. We could check the Hamiltonian' eigenvalues with experimental data or compare to other computations from https://cccbdb.nist.gov/energy2.asp 
However, in the example the molecule integrals are provided by OpenFermion only for $1,45 \mathring{A}$. If you look up the experimental geometry (correct for the Born-Openheimer approximation), $r_{LiH} = 1.595 \mathring{A}$ for $^7Li$ https://cccbdb.nist.gov/expgeom2.asp.

You can see that the matrix calculation would result in exponential runtimes for larger systems. We convert the Hamiltonian for simulation with a quantum computer into the so-called "second quantized" operator form, as was shown in Tutorial II.
$$
H = \sum_{pq} T_{pq} a^\dagger_p a_q + \sum_{pq} V_{pq} a^\dagger_p a_p a^\dagger_q a_q.
$$

In [2]:
fermion_operator = openfermion.get_fermion_operator(molecular_hamiltonian)
print("Fermionic Hamiltonian with {} terms".format( len(fermion_operator.terms)))
print(fermion_operator)

Fermionic Hamiltonian with 73 terms
-6.7698132180879735 [] +
-0.7952726864779313 [0^ 0] +
0.24889540266275176 [0^ 0^ 0 0] +
-0.02307282640154995 [0^ 0^ 0 2] +
-0.023072826401549944 [0^ 0^ 2 0] +
0.005865992881900444 [0^ 0^ 2 2] +
0.24889540266275176 [0^ 1^ 1 0] +
-0.02307282640154995 [0^ 1^ 1 2] +
-0.023072826401549944 [0^ 1^ 3 0] +
0.005865992881900444 [0^ 1^ 3 2] +
0.04614563473199314 [0^ 2] +
-0.02307282640154995 [0^ 2^ 0 0] +
0.005865992881900456 [0^ 2^ 0 2] +
0.11412688446849813 [0^ 2^ 2 0] +
0.0027487522157917266 [0^ 2^ 2 2] +
-0.02307282640154995 [0^ 3^ 1 0] +
0.005865992881900456 [0^ 3^ 1 2] +
0.11412688446849813 [0^ 3^ 3 0] +
0.0027487522157917266 [0^ 3^ 3 2] +
0.24889540266275176 [1^ 0^ 0 1] +
-0.02307282640154995 [1^ 0^ 0 3] +
-0.023072826401549944 [1^ 0^ 2 1] +
0.005865992881900444 [1^ 0^ 2 3] +
-0.7952726864779313 [1^ 1] +
0.24889540266275176 [1^ 1^ 1 1] +
-0.02307282640154995 [1^ 1^ 1 3] +
-0.023072826401549944 [1^ 1^ 3 1] +
0.005865992881900444 [1^ 1^ 3 3] +
-0.023072826

We see from the above output that this is a fairly complex Hamiltonian already. Next we will use the `simulate_trotter` function from Tutorial I, but this time with the built-in `LOW_RANK` Trotter step type, associated with these low rank techniques.

Setup the simulation environment

In [3]:
import cirq
import openfermioncirq
from openfermioncirq import trotter

# Trotter step parameters.
time = 1.
final_rank = 2

# Initialize circuit qubits in a line.
n_qubits = openfermion.count_qubits(molecular_hamiltonian)
qubits = cirq.LineQubit.range(n_qubits)

In the cell below, we compile the Trotter step with full rank so $L = N^2$ and depth is actually $O(N^3)$ and repeat the Trotter step multiple times to show that it actually converges to the correct result. Note that the rank of the Coulomb operators is asymptotically $O(N)$ but for very small molecules in small basis sets only a few eigenvalues can be truncated.

In [4]:
# Initialize a random initial state.
import numpy
random_seed = 8317
initial_state = openfermion.haar_random_vector(
    2 ** n_qubits, random_seed).astype(numpy.complex64)

# Trotter step paramaters.
n_steps = 3

# Compile the low rank Trotter step using OpenFermion-Cirq.
qubits = cirq.LineQubit.range(n_qubits)
circuit = cirq.Circuit(
    trotter.simulate_trotter(
            qubits, molecular_hamiltonian,
            time=time, n_steps=n_steps,
            algorithm=trotter.LOW_RANK),
    strategy=cirq.InsertStrategy.EARLIEST)

# Print circuit.
cirq.DropNegligible().optimize_circuit(circuit)
print(circuit.to_text_diagram(transpose=True))

0             1                     2                3
│             │                     │                │
Rz(π)         Rz(π)                 Rz(π)            Rz(π)
│             │                     │                │
│             PhISwap(0.25)─────────PhISwap(0.25)^-1 │
│             │                     │                │
PhISwap(0.25)─PhISwap(0.25)^0.081   │                │
│             │                     │                │
Rz(0.112π)    │                     PhISwap(0.25)────PhISwap(0.25)^-0.081
│             │                     │                │
│             PhISwap(0.25)─────────PhISwap(0.25)^-1 │
│             │                     │                │
│             Rz(0.112π)            │                Rz(0.056π)
│             │                     │                │
│             │                     Rz(0.056π)       │
│             │                     │                │
│             │                     │                │
│             │                  

For comparison we compute the time step with the exact time evolution operator $\psi(t) = e^{ -i H t } \psi(0)$ in matrix form.

In [5]:
# Numerically compute the correct circuit output.
import scipy
exact_state = scipy.sparse.linalg.expm_multiply(
    -1j * time * hamiltonian_sparse, initial_state)

In [6]:
%matplotlib
import matplotlib.pyplot as plt
fig, axs = plt.subplots(2)
axs[1].set_xlabel('basis')
axs[0].set_ylabel('Re(psi)')
axs[1].set_ylabel('Im(psi)')
axs[0].set_title('Simulation')
x = range(exact_state.size)
axs[0].plot(x, exact_state.real)
axs[1].plot(x, -exact_state.imag)

Using matplotlib backend: MacOSX


Compare to complex conjugate state, why?
And why are Re and Im exchanged between exact and simulated?
Looks as if a phase factor is there...

In [7]:
# Use Cirq simulator to apply circuit.
simulator = cirq.Simulator()
result = simulator.simulate(circuit, qubit_order=qubits, initial_state=initial_state)
simulated_state = result.final_state
print( result )
axs[1].plot(x, simulated_state.real, dashes=[1, 1])
axs[0].plot(x, simulated_state.imag, dashes=[1, 1])

# Print final fidelity.
fidelity = abs(numpy.dot(simulated_state, numpy.conjugate(exact_state))) ** 2
print('Fidelity with exact result is {}.\n'.format(round(fidelity, 6)))


measurements: (no measurements)
output vector: [ 0.03731383+0.05890184j  0.13437001-0.04669848j  0.05842413+0.24252099j
  0.00585651+0.15398595j  0.03262561-0.5150042j   0.25542358-0.17139976j
 -0.06184286+0.06148679j -0.08049247-0.02335707j -0.15040077+0.03903422j
  0.30606395+0.05003889j -0.16854814+0.18526214j -0.19785163+0.071437j
  0.25791448-0.38240576j -0.01672529+0.12084839j -0.1230133 +0.11979542j
  0.10457645+0.1342528j ]
Fidelity with exact result is 0.999996.



In [8]:
# Compute next time step
exact_state = scipy.sparse.linalg.expm_multiply(
    -1j * time * hamiltonian_sparse, exact_state)
axs[0].plot(x, exact_state.real)
axs[1].plot(x, -exact_state.imag)

In [9]:
# Run simulator again on simulated state
result = simulator.simulate(circuit, qubit_order=qubits, initial_state=simulated_state)
simulated_state = result.final_state
axs[0].plot(x, -simulated_state.real, dashes=[1, 1])
axs[1].plot(x, simulated_state.imag, dashes=[1, 1])
fidelity = abs(numpy.dot(simulated_state, numpy.conjugate(exact_state))) ** 2
print('Fidelity with exact result is {}.\n'.format(round(fidelity, 6)))

Fidelity with exact result is 0.999985.



In [10]:
for i, step in enumerate(simulator.simulate_moment_steps(circuit)):
    print('state at step %d: %s' % (i, numpy.around(step.state_vector(), 3)))
    #plt.plot(x, numpy.real(step.state_vector()))
    #plt.plot(x, numpy.imag(step.state_vector()))

plt.show()

state at step 0: [1.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j
 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
state at step 1: [1.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j
 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
state at step 2: [1.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j
 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
state at step 3: [0.984-0.176j 0.   +0.j    0.   +0.j    0.   +0.j    0.   +0.j
 0.   +0.j    0.   +0.j    0.   +0.j    0.   +0.j    0.   +0.j
 0.   +0.j    0.   +0.j    0.   +0.j    0.   +0.j    0.   +0.j
 0.   +0.j   ]
state at step 4: [0.984-0.176j 0.   +0.j    0.   +0.j    0.   +0.j    0.   +0.j
 0.   +0.j    0.   +0.j    0.   +0.j    0.   +0.j    0.   +0.j
 0.   +0.j    0.   +0.j    0.   +0.j    0.   +0.j    0.   +0.j
 0.   +0.j   ]
state at step 5: [0.904-0.427j 0.   +0.j    0.   +0.j    0.   +0.j    0.   +0.j
 0.   +0.j    0.   +0.j    0.   +0.j    0.   +0.j    0.   +0.j
 0.   +0.j    0.   

 0.   +0.j   ]
state at step 41: [0.926-0.379j 0.   +0.j    0.   +0.j    0.   +0.j    0.   +0.j
 0.   +0.j    0.   +0.j    0.   +0.j    0.   +0.j    0.   +0.j
 0.   +0.j    0.   +0.j    0.   +0.j    0.   +0.j    0.   +0.j
 0.   +0.j   ]
state at step 42: [0.926-0.379j 0.   +0.j    0.   +0.j    0.   +0.j    0.   +0.j
 0.   +0.j    0.   +0.j    0.   +0.j    0.   +0.j    0.   +0.j
 0.   +0.j    0.   +0.j    0.   +0.j    0.   +0.j    0.   +0.j
 0.   +0.j   ]
state at step 43: [0.926-0.379j 0.   +0.j    0.   +0.j    0.   +0.j    0.   +0.j
 0.   +0.j    0.   +0.j    0.   +0.j    0.   +0.j    0.   +0.j
 0.   +0.j    0.   +0.j    0.   +0.j    0.   +0.j    0.   +0.j
 0.   +0.j   ]
state at step 44: [0.926-0.379j 0.   +0.j    0.   +0.j    0.   +0.j    0.   +0.j
 0.   +0.j    0.   +0.j    0.   +0.j    0.   +0.j    0.   +0.j
 0.   +0.j    0.   +0.j    0.   +0.j    0.   +0.j    0.   +0.j
 0.   +0.j   ]
state at step 45: [0.926-0.379j 0.   +0.j    0.   +0.j    0.   +0.j    0.   +0.j
 0.   +0.j    0.